In [1]:
import requests
from bs4 import BeautifulSoup
import newspaper
from newspaper import Article

# user-agent
request_header = {
    "User-Agent":
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 "
        "Safari/537.36 Edg/114.0.1823.58",
    "Connection":
        "keep-alive"
    # "Referer":
    #     "https://www.baidu.com"
}

In [2]:
# 保存到txt文件
def dump_news(news_title, news_content):
    # 指定路径为"data"目录
    with open(f"data/{news_title}.txt", "w", encoding="utf-8") as f:
        f.write(news_content)

## 操作mysql数据库

In [2]:
import pymysql
# 创建连接
cnx = pymysql.connect(host='localhost', user='user', password='passport')
cursor = cnx.cursor()

def create_database():
    # 创建数据库
    cursor.execute("CREATE DATABASE IF NOT EXISTS IR_db")
    # 使用数据库
    cnx.select_db('IR_db')

def create_table():
    # 创建表
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS html (
            id INT AUTO_INCREMENT PRIMARY KEY,
            title VARCHAR(255),
            content TEXT,
            url VARCHAR(255),
            date DATETIME
        )
    """)
    # 提交
    cnx.commit()

create_database()
create_table()

In [3]:
from datetime import datetime

# 保存到mysql数据库：id title content url
def dump_to_mysql(title, content, url, date):
    global unique_id
    # 查询url是否存在
    cursor.execute(f"SELECT * FROM html WHERE url='{url}'")
    # 如果不存在则插入
    if cursor.fetchone() is None:
        dt = datetime.strptime(date, "%Y-%m-%d")
        query = f"INSERT INTO html (title, content, url, date) VALUES ('{title}', '{content}', '{url}', '{dt}')"
        cursor.execute(query)
        cnx.commit()
        print(f"插入成功：{title}")
    else:
        print(f"已存在：{title}")
    

In [8]:
# test
url = "https://news.southcn.com/node_179d29f1ce/aa4be69a91.shtml"
article = Article(url, language='zh')
article.download()
article.parse()
article.nlp()
dump_to_mysql(article.title, article.text, url)

插入成功：中美两国发表关于加强合作应对气候危机的阳光之乡声明


### 清空表

In [18]:
# 清空表中内容
def clear_table():
    cursor.execute("TRUNCATE TABLE html")
    cnx.commit()

clear_table()

## 操作neo4j数据库

In [ ]:
# 保存到neo4j数据库：id title url 链接关系
def dump_to_neo4j(id, title, url):
    pass

## 尝试使用newspaper处理外链

In [6]:
# 保存到mysql数据库
def try_external_link(url):
    # 尝试从外部链接中获取内容
    article = Article(url)
    try:
        article.download()
        article.parse()
        return article.title, article.text
    except:
        print(f"获取失败：{url}")
        return '', ''

In [8]:
# test
# 尝试从外部链接中获取内容
article = Article('https://baijiahao.baidu.com/s?id=1753635137351175023')
try:
    article.download()
    article.parse()
    print(len(article.title) , article.title)
    print(len(article.text), article.text)
except:
    print(f"获取失败：{url}")

Building prefix dict from d:\pythonFile\python3.7\lib\site-packages\jieba\dict.txt ...
Loading model from cache C:\Projects\Temp\jieba.cache
Loading model cost 0.864976167678833 seconds.
Prefix dict has been built succesfully.


6 百度安全验证
0 


## 爬睡前消息新闻列表

In [19]:
import re

def get_daily_news():
    url = 'https://archive.bedtime.news/zh/daily/'
    for i in range(2022,2024):
        year = str(i)+'.md'
        year_url = url + year
        resp = requests.get(year_url, headers=request_header)
        html_text = resp.text
        soup = BeautifulSoup(html_text, 'lxml')
        # 获取每个月的新闻列表
        month_lists = soup.select('div.tabset-panel > ul.links-list')
        #print(month_lists)
        # 获取每天的新闻链接
        for month_list in month_lists:
            day_lists = month_list.select('li')
            for day_list in day_lists:
                day_a = day_list.select('a')[0]
                day_url = day_a.get('href').strip()
                day_url = url + day_url.lstrip('/.')
                get_daily_content(day_url)

def remove_prefix(text):
    return re.sub(r'^h-\d+-','',text)

def get_p_text(html_content):
    # 用BeautifulSoup解析HTML内容
    soup = BeautifulSoup(html_content, 'html.parser')
    # 选择<p>标签，并提取文本内容
    p_tag = soup.find('p')
    try:
        text_inside_p = p_tag.text
        return text_inside_p
    except:
        return ''

def get_next_title(tag):
    next_tag = tag.find_next_sibling()
    if next_tag.name == 'h3':
        return False, next_tag.get('id').strip()
    else:
        return True, get_p_text(next_tag.__str__())

def get_p_between_h2s(h2_tag):
    next_tags = h2_tag.find_next_siblings()
    p_tags = []
    for tag in next_tags:
        if tag.name == 'h2':
            break
        if tag.name == 'p':
            p_tags.append(tag)
    return p_tags

def get_daily_content(day_url):
    resp = requests.get(day_url, headers=request_header)
    html_text = resp.text
    soup = BeautifulSoup(html_text, 'lxml')
    news_list = soup.select('h2.toc-header')  # 新闻列表
    if news_list != []:
        date = soup.select('page')[0].get('title').strip()  # 日期
    for news in news_list:
        # 获取新闻标题
        news_title = news.get('id').strip()
        news_title = remove_prefix(news_title)
        # 新闻内容由很多p组成
        ps = get_p_between_h2s(news)
        all_text = []
        for i in range(len(ps)):
            p = ps[i]
            # 检查是否有外部链接
            if p.find('a'):
                # 有外部链接，前面的构成一个新闻
                a = p.find('a')
                url = a.get('href')
                news_content = '\n'.join(all_text)
                # 尝试下能不能从url提取内容 ×太慢了，效果也一般
                # temp_title, temp_content = try_external_link(url)
                # if temp_title != '' and temp_content != '' and len(temp_content) > len(news_content):
                #     news_title = temp_title
                #     news_content = temp_content
                # 保存到mysql数据库
                try:
                    dump_to_mysql(news_title, news_content, url, date)
                except:
                    print(f"保存失败：{url}")
                # 为下一新闻做准备
                all_text = []
                # 如果是末尾的，直接退出了
                if i == len(ps)-1:
                    break
                # 如果不是末尾的，将下一个p或者h3作为新闻标题
                isP, news_title = get_next_title(p)
                if isP:
                    i += 1
            else:
                # 直接将p中的内容加入到新闻内容中
                text = get_p_text(p.__str__())
                all_text.append(text)      
        


In [20]:
get_daily_news()

插入成功：越南两位副总理辞去党内职务
插入成功：核酸腐败窝案之后越南疫情包机也出事了外长受处分
插入成功：山东两地放开烟花禁令
插入成功：时近春节，关于烟花爆竹“禁燃令”是否松绑，再一次成为舆论的热点话题
插入成功：上海警方抓获20多名殡葬黄牛通过插队得到名额加价卖给市民
插入成功：新冠感染医保能报销吗怎么报多地明确
插入成功：驻英国大使中国近期优化调整防控措施建立在科学评估之上
插入成功：地铁公司悄悄成为拿地新势力
插入成功：12月官方制造业pmi录得47-为2020年3月以来最低
插入成功：全球集装箱溢出流入中国有码头空箱堆存量占比超90
插入成功：中纪委委员王晓萍任人社部部长
插入成功：77岁昂山素季又一案件宣判-累计刑期已达33年
插入成功：我国热点论文数量首次升至世界第一
插入成功：南水北调集团列入国资委监管央企户数增至98家
插入成功：美国埃克森美孚起诉欧盟征收暴利税越权
插入成功：普京签署总统令允许不友好国家买家使用外币支付俄罗斯天然气欠款
插入成功：俄航向爱尔兰租赁商购买10架波音777客机
插入成功：美国2022年对北约成员国军售几乎翻倍
插入成功：无证据质疑李轩豪九段作弊-中国围棋国家队队员杨鼎新九段被禁赛半年
插入成功：37岁c罗正式加盟沙特球队签约至2025年年薪超2亿美元
插入成功：驻俄大使张汉晖就苏联成立100周年接受rt书面专访
插入成功：中俄领导人视频对话
插入成功：海南三亚-海口整体已过感染高峰全省预计感染率已达50
插入成功：中国应急批准默沙东新冠口服药
插入成功：欧盟卫生官员反对向中国旅客实施限制措施
插入成功：谭德塞称多国对中国入境旅客施加限制措施可理解外交部回应
插入成功：多国发微博欢迎中国游客
插入成功：江歌案二审维持原判-刘鑫认为自己无过错
插入成功：证大系集资诈骗案一审宣判戴志康被判十九年处罚金2550万元
插入成功：2023年全国两会召开时间确定
插入成功：中国驻美国大使秦刚任外交部部长
插入成功：美rc135侦察机在南海被解放军拦截称中国战机不安全
插入成功：白俄罗斯在领空内击落乌导弹召见乌大使提出抗议
插入成功：塞尔维亚总统解除武装部队最高级别战备状态令
插入成功：天津津门虎-北京国安弃赛-武汉三镇夺得中超冠军
插入成功：巴西球王贝利去世-享年82岁
插入成功：工信部布洛芬和对乙酰氨基酚日产量19亿片较月初翻超4

### 数据清洗

In [21]:
# 数据清洗，把内容为空的添加他的标题进去
def update_empty_content():
    # 找到所有content字段为空或只包含空白字符（包括换行符）的行
    cursor.execute("SELECT id, title FROM html WHERE TRIM(REPLACE(content, '\n', '')) IS NULL OR TRIM(REPLACE(content, '\n', '')) = ''")
    rows = cursor.fetchall()
    # 使用标题来更新这些行的content字段
    for row in rows:
        id, title = row
        cursor.execute("UPDATE html SET content = %s WHERE id = %s", (title, id))
    # 提交更改
    cnx.commit()

# 数据清洗，把爬错的删除
def delete_wrong_title_row():
    # 删除那些title字段等于url字段的行
    cursor.execute("DELETE FROM html WHERE title = url")
    # 提交更改
    cnx.commit()

# 数据清洗，把标题为空的添加他的内容进去
def update_empty_title():
    # 找到所有title字段为空的行
    cursor.execute("SELECT id, content FROM html WHERE title IS NULL OR title = ''")
    rows = cursor.fetchall()
    # 使用content的前五个字符来更新这些行的title字段
    for row in rows:
        id, content = row
        new_title = content[:5]  # 获取content的前五个字符
        cursor.execute("UPDATE html SET title = %s WHERE id = %s", (new_title, id))
    # 提交更改
    cnx.commit()

In [22]:
update_empty_content()
delete_wrong_title_row()
update_empty_title()

### test

In [32]:
def print_news_content(id):
    # 获取id为指定值的新闻内容
    cursor.execute("SELECT content FROM html WHERE id = %s", (id,))
    row = cursor.fetchone()
    # 打印新闻内容
    if row is not None:
        print(repr(row[0]))  # 使用repr函数可以打印出原始字符串，包括其中的空白字符
    else:
        print("没有找到id为{}的新闻".format(id))

# 调用函数
print_news_content(1547)

'在韩表演僵尸跆舞跆拳道馆被处罚丑化民族形象亵渎中华文化造成恶劣影响'
